# Testing operations with PyTorch

In [1]:
import numpy as np
import torch
import random
import parameters as var #Configuration and coarsening parameters
var.init() #initializes parameters
import utils as ut #Some utility functions 
import loss_function as lf #Custom loss function


from opendataset import ConfsDataset #class for opening gauge confs
from opendataset import read_binary_conf
import operators as op #Interpolator and prolongator given a set of test vectors
import operators_torch as opt #Same implementations but with pytorch

In [2]:
def numpy_loss(pred_test_vectors,test_vectors):
    loss = 0
    for i in range(batch_size):
        ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors[i])
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[i,tv] - ops.P_Pdagg(test_vectors[i,tv])) )
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    return loss

def torch_loss(pred, target):
    """
    pred  : Tensor of shape (B, NV, 2, NT, NX)   (complex numbers stored as complex dtype)
    target: Tensor of shape (B, NV, 2, NT, NX)   (the “near kernel” vectors)
    Returns a scalar loss Tensor (requires_grad=True)
    """
    batch_size = pred.shape[0]
    loss = 0.0
    for i in range(batch_size):
            
        ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred[i])   
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[i, tv])               
            diff = target[i, tv] - corrected
            loss = loss + torch.linalg.norm(diff)   
    return loss

In [3]:
def numpy_loss_ind(pred_test_vectors,test_vectors):
    loss = 0.0
    ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors)
    if test_vectors.shape[0] == var.NV:
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[tv] - ops.P_Pdagg(test_vectors[tv])) )
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    else:
        loss += np.linalg.norm( (test_vectors - ops.P_Pdagg(test_vectors)) )
    return loss

def torch_loss_ind(pred, target):
    loss = 0.0      
    ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred)
    if target.shape[0] == var.NV:
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[tv])               
            diff = target[tv] - corrected
            loss = loss + torch.linalg.norm(diff)
    else:
        loss += torch.linalg.norm( (target - ops.P_Pdagg(target)) )  
    return loss

In [4]:
"""
Loading the configurations and the near-kernel test vectors
"""
workers = 4
batch_size = 300
dataset = ConfsDataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                shuffle=False, num_workers=workers)

#----returns a tensor of size [ [batch_size,4,Nx,Nt], [batch_size,Nv,2,Nx,Nt] ]----#
first_batch = next(iter(dataloader)) 
#--------------------------------------

In [5]:
#For testing torch vs numpy
test = first_batch[1][0]

print("set of test vectors",type(test))
oper = op.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs = oper.getTestVectors()
print("numpy vectors",type(tvecs))
oper_torch = opt.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs_torch = oper_torch.getTestVectors()
print("torch vectors",type(tvecs_torch))
print(tvecs.shape,tvecs_torch.shape)
print("test vectors",tvecs[0,1,2,2],tvecs_torch[0,1,2,2])

v = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
vc = np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X) + 1j*np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X)

vtorch = torch.tensor(v)
vctorch = torch.tensor(vc)


out = oper.P_vc(vc)
print("Pvc (np)",out[0,5,0],type(out))
outt = oper_torch.P_vc(vctorch)
print("Pvc (torch)",outt[0,5,0],type(outt))

out = oper.P_Pdagg(v)
print("PP^+ (np)",v[0,5,0],out[0,5,0])
outvc = oper.Pdagg_P(vc)
print("P^+P (np)",vc[0,0,0,0],outvc[0,0,0,0])

outt = oper_torch.P_Pdagg(vtorch)
print("PP^+ (torch)",vtorch[0,5,0],outt[0,5,0])
outvct = oper_torch.Pdagg_P(vctorch)
print("P^+P (torch)",vctorch[0,0,0,0],outvct[0,0,0,0])


set of test vectors <class 'torch.Tensor'>
numpy vectors <class 'numpy.ndarray'>
torch vectors <class 'torch.Tensor'>
(14, 2, 32, 32) torch.Size([14, 2, 32, 32])
test vectors (0.008464727400800228+0.0023593841878715963j) tensor(0.0085+0.0024j, dtype=torch.complex128)
Pvc (np) (-0.2757831186377956-0.23684182183238545j) <class 'numpy.ndarray'>
Pvc (torch) tensor(-0.2758-0.2368j, dtype=torch.complex128) <class 'torch.Tensor'>
PP^+ (np) (0.4242191196728845+0.5110749252893962j) (0.053702916291525696-0.00923173486874837j)
P^+P (np) (0.9436472982993661+0.25456799570715427j) (0.9436472982993656+0.2545679957071538j)
PP^+ (torch) tensor(0.4242+0.5111j, dtype=torch.complex128) tensor(0.0537-0.0092j, dtype=torch.complex128)
P^+P (torch) tensor(0.9436+0.2546j, dtype=torch.complex128) tensor(0.9436+0.2546j, dtype=torch.complex128)


In [6]:
test_vectors = np.zeros((var.NV,2,var.NT,var.NX),dtype=complex)
#random.seed(0)
for tv in range(var.NV):
    for nt in range(var.NT):
        for nx in range(var.NX):
            for s in range(2):
                x = random.random()
                y = random.random()
                z = complex(x, y)
                test_vectors[tv,s,nt,nx] = z   #2*(nx*NT + nt) + s + 1 + tv*2*NX*NT

In [7]:
print(numpy_loss_ind(first_batch[1][0].numpy(),first_batch[1][0].numpy()))
print(torch_loss_ind(first_batch[1][0],first_batch[1][0]))

8.851622487373178e-14
tensor(8.7847e-14, dtype=torch.float64)


In [17]:
#This checks whether random test vectors are contained on the image of P when this operator is 
#assembled with the test vectors from SAP
print(numpy_loss_ind(first_batch[1][0].numpy(),test_vectors)/var.NV)
print(torch_loss_ind(first_batch[1][0],torch.tensor(test_vectors))/var.NV)

35.931581442847964
tensor(35.9316, dtype=torch.float64)


In [16]:
#This does the opposite: checks whether the test vectors from SAP are contained in the image of P when built
#with random vectors
print(numpy_loss_ind(test_vectors,first_batch[1][0].numpy())/var.NV)
print(torch_loss_ind(torch.tensor(test_vectors),first_batch[1][0])/var.NV)

9.799700947722286
tensor(9.7997, dtype=torch.float64)


In [9]:
#Checking that the test vectors generated with SAP are approximately contained in the range of P
loss = 0.0
remTV = 15-var.NV
for confID in range(batch_size):
    print("confID",confID)
    for i in range(remTV):
        tvector = np.zeros((2,var.NT,var.NX),dtype=complex)
        path = 'confs/near_kernel/b{0}_{1}x{2}/{3}/tvector_{1}x{2}_b{0}0000_m{4}_nconf{5}_tv{6}.tv'.format(
        int(var.BETA),var.NX,var.NT,var.M0_FOLDER,var.M0_STRING,confID,14-i)
        tvector = read_binary_conf(None,path)
        print("Test vector ",14-i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*remTV))

confID 0
Test vector  14
Numpy 1.9623770752968441
PyTorch tensor(1.9624, dtype=torch.float64)
----------------------
confID 1
Test vector  14
Numpy 1.8214149300023192
PyTorch tensor(1.8214, dtype=torch.float64)
----------------------
confID 2
Test vector  14
Numpy 1.5564233443395599
PyTorch tensor(1.5564, dtype=torch.float64)
----------------------
confID 3
Test vector  14
Numpy 2.039277328068034
PyTorch tensor(2.0393, dtype=torch.float64)
----------------------
confID 4
Test vector  14
Numpy 1.840542668015146
PyTorch tensor(1.8405, dtype=torch.float64)
----------------------
confID 5
Test vector  14
Numpy 1.5851264189150664
PyTorch tensor(1.5851, dtype=torch.float64)
----------------------
confID 6
Test vector  14
Numpy 1.5969146051067047
PyTorch tensor(1.5969, dtype=torch.float64)
----------------------
confID 7
Test vector  14
Numpy 1.6488358732922765
PyTorch tensor(1.6488, dtype=torch.float64)
----------------------
confID 8
Test vector  14
Numpy 1.8171664952989297
PyTorch tensor(1

Final loss on the remainder test vectors 9.386980374900652 (for 4 test vectors and using the rest to check the loss function)
Final loss on the remainder test vectors 2.5979268393039137 (for 14 test vectors and using the rest to check the loss function)

This shows that the more smoothed test vectors, the better they capture other near-kernel components.

Just out of curiosity. What happens if I evaluate the metric just on random vectors?

In [12]:
#Using the test vectors from SAP to build P, P^+ and evaluate on random test vectors.
loss = 0.0
for confID in range(batch_size):
    print("confID",confID)
    for i in range(var.NV):
        tvector = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
        print("rand vector",i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*var.NV))

confID 0
rand vector  0
Numpy 35.58058936175008
PyTorch tensor(35.5806, dtype=torch.float64)
rand vector  1
Numpy 36.132749784750345
PyTorch tensor(36.1327, dtype=torch.float64)
rand vector  2
Numpy 35.880615979164816
PyTorch tensor(35.8806, dtype=torch.float64)
rand vector  3
Numpy 36.0268933181431
PyTorch tensor(36.0269, dtype=torch.float64)
rand vector  4
Numpy 36.20219203073775
PyTorch tensor(36.2022, dtype=torch.float64)
rand vector  5
Numpy 35.97462777620213
PyTorch tensor(35.9746, dtype=torch.float64)
rand vector  6
Numpy 36.32895399377162
PyTorch tensor(36.3290, dtype=torch.float64)
rand vector  7
Numpy 36.16603575723268
PyTorch tensor(36.1660, dtype=torch.float64)
rand vector  8
Numpy 36.039900646875815
PyTorch tensor(36.0399, dtype=torch.float64)
rand vector  9
Numpy 36.331295981763574
PyTorch tensor(36.3313, dtype=torch.float64)
rand vector  10
Numpy 35.69556898958965
PyTorch tensor(35.6956, dtype=torch.float64)
rand vector  11
Numpy 36.080864462500564
PyTorch tensor(36.0809

In [ ]:
loss = 0.0
for confID in range(batch_size):
    print("confID",confID)
    for i in range(var.NV):
        tvector = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
        print("rand vector",i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*var.NV))